# Weedle's Cave challenge in R 
## Predicting wins and losses between pokémon in Gen 1

### This is "Weedle's Cave" challenge. Can we build a model that'll predict which pocket monster will win or lose?

##### Personal objectives: Get familiar with R.

###### First, Read in the data! We'll read in the pokémon list, the combats that we'll model from, and the final test data that we need to populate for our Kaggle submission. Should be self explanatory, but the super high-level workflow is as follows:

1. Profile/clean/investigate the data
2. Build some models and see what works well
3. Apply the final chosen model to our test data for our Kaggle submission

In [18]:
# first row contains variable names, comma is separator 
# assign the variable id to row names
# note the / instead of \ on mswindows systems 

data_pokemon <- read.csv("/Users/kintesh/Documents/kaggle/pokemon/data/pokemon.csv")
data_combats <- read.csv("/Users/kintesh/Documents/kaggle/pokemon/data/combats.csv")
data_tests <- read.csv("/Users/kintesh/Documents/kaggle/pokemon/data/tests.csv")
# mydata <- read.table("/Users/kintesh/Documents/kaggle/pokemon/data/pokemon.csv", header=TRUE, 
#    sep=",", row.names="id", quote="")

In [19]:
head(data_pokemon,5)
head(data_combats,5)
head(data_tests,5)

X.,Name,Type.1,Type.2,HP,Attack,Defense,Sp..Atk,Sp..Def,Speed,Generation,Legendary
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
5,Charmander,Fire,,39,52,43,60,50,65,1,False


First_pokemon,Second_pokemon,Winner
266,298,298
702,701,701
191,668,668
237,683,683
151,231,151


First_pokemon,Second_pokemon
129,117
660,211
706,115
195,618
27,656
